In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from collections import OrderedDict
sns.set_style("whitegrid", {'axes.grid' : False})

import pyspark.sql.functions as F
from pyspark.sql.types import *
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import Row
import pyarrow.parquet as pq
import pyarrow as pa
import os


os.environ['SPARK_HOME'] = "/home/veselovs/spark-3.2.1-bin-hadoop2.7"
os.environ['JAVA_HOME'] = "/home/veselovs/jdk-13.0.2"
spark = SparkSession.builder.getOrCreate()


22/10/19 16:39:44 WARN Utils: Your hostname, iccluster039 resolves to a loopback address: 127.0.1.1; using 10.90.38.15 instead (on interface ens786f0)
22/10/19 16:39:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/19 16:39:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/19 16:39:45 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/10/19 16:39:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/19 16:39:46 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [16]:
full = spark.read.load("/scratch/venia/web2wiki/data/en_wikilinks_clean.parquet/")

In [38]:
full.groupBy("domain", "title").count().write.parquet("/scratch/venia/web2wiki/data/en_wikilinks_domain_counts.parquet/", mode = "overwrite")

In [42]:
df = spark.read.load("/scratch/venia/web2wiki/data/en_wikilinks_domain_counts.parquet/")

In [43]:
df2 = spark.read.load("/scratch/venia/web2wiki/data/en_wikilinks_url_counts.parquet")

# How many URLS, average wikis / url, max wikis per url. 

In [47]:
print("How many shares", df.agg(F.sum("count")).collect())
print("Most wikis in one page", df.agg(F.max("count")).collect())
print("Mean wikis in one page", df.agg(F.mean("count")).collect())
print("unique number domains", df.agg(F.countDistinct("domain")).collect())

How many shares [Row(sum(count)=48829702)]
Most wikis in one page [Row(max(count)=1056787)]
Mean wikis in one page [Row(avg(count)=3.099593406782444)]


unique number domains [Row(count(domain)=940239)]


In [48]:
print("How many shares", df2.agg(F.sum("count")).collect())
print("Most wikis in one page", df2.agg(F.max("count")).collect())
print("Mean wikis in one page", df2.agg(F.mean("count")).collect())
print("unique number domains", df2.agg(F.countDistinct("url")).collect())

How many shares [Row(sum(count)=48829702)]
Most wikis in one page [Row(max(count)=4368)]
Mean wikis in one page [Row(avg(count)=1.2206566768027869)]


unique number domains [Row(count(url)=14462267)]


In [57]:
df.groupBy("title").agg(F.sum(F.col("count"))).orderBy("sum(count)",ascending=False).take(10)

[Row(title='', sum(count)=1125788),
 Row(title='United States', sum(count)=154211),
 Row(title='HTTP cookie', sum(count)=142842),
 Row(title='Wikipedia:Text of Creative Commons Attribution-ShareAlike 3.0 Unported License', sum(count)=107867),
 Row(title='Main Page', sum(count)=92899),
 Row(title='Copyright law of the United States', sum(count)=63225),
 Row(title='Philippine Amusement and Gaming Corporation', sum(count)=63166),
 Row(title='California', sum(count)=62366),
 Row(title='Wikipedia:Contact us', sum(count)=59351),
 Row(title='Digital object identifier', sum(count)=59153)]